In [2]:
import pandas as pd
import re
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
from collections import Counter
import datetime

In [3]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [4]:
import json

def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [5]:
### Define the data cleaning functions for all collected data. 

### omdb_movie_data_list.json

def get_rt_score(movie_data):
    """Get the Rotten Tomatoes score for a movie from omdb_movie_data dictionary."""
    ratings = movie_data.get("Ratings")
    if ratings is not None:
        for rating in ratings:
            if rating["Source"] == "Rotten Tomatoes":
                return rating["Value"]
    return np.nan

def split_long_string(string):
    """Split a long string of names into a list of names."""
    split_string = string.split(", ")
    return split_string

def clean_runtime(runtime):
    """Clean movie runtime values."""
    runtime = runtime.strip(" min") 
    return runtime

def clean_country(country_list):
    """Clean the country values for each movie."""
    country_list = ['USA' if country=='United States' else country for country in country_list] 
    return country_list

def clean_imdbvotes(votes):
    "Clean imdbvotes values."
    votes = re.sub(",", "", votes)
    return votes

def clean_boxoffice(boxoff_val):
    """Clean box office values."""
    if boxoff_val is not None:
        boxoff_val = re.sub("[$,]", "", boxoff_val)
    return boxoff_val

def clean_writers(writers):
    """Clean writers string by removing their roles in parentheses after name.""" 
    writers = re.sub("(.\([a-zA-Z\s]+\))", "", writers)
    return writers

### tmdb_movie_data_list.json

def append_release_dates(movie_data):
    """Extract the nested theatrical release date/s for a movie, append to the first level of the movie_data dictionary; return movie_data."""

    if movie_data["Release_Dates"] is not None:
        for release in movie_data["Release_Dates"]: 
            country_code = release.get("iso_3166_1")
            for data in release.get("release_dates"):
                if data["type"] == 3:
                    release_date = data.get("release_date")
                    movie_data[f'{country_code}_release_date'] = release_date
    return movie_data

def append_keywords(movie_data):
    """Extract the nested keywords for a movie and append a list to the first level of the movie_data dictionary; return movie_data."""
    keyword_list = []
    if (movie_data["Keywords"] is not None) and (len(movie_data["Keywords"]) != 0):
        for keyword in movie_data["Keywords"]:
            keyword = keyword.get("name")
            keyword_list.append(keyword)
    movie_data["Keyword_List"] = keyword_list
    return movie_data

### golden_globe_awards.csv / the_oscar_award.csv 

def convert_award_date(year_award):
    """Add the month to the year and convert to datetime object."""
    year_award = pd.to_datetime(year_award + "-04")
    return year_award

### the_grammy_awards.csv

def clean_grammy_workers(workers_string):
    """Clean the workers column of the grammy awards dataframe and return a list of names."""
    workers_string = re.sub("(\w*\s*\w+;)", "", workers_string) # remove words followed by ";" e.g. "conductor;".
    workers_string = re.sub("&", ",", workers_string) 
    workers_string = re.sub("(\weaturing)", ",", workers_string)
    workers_string = re.sub("(\(\w\))", "", workers_string) # remove "(A)" and "(T)"
    workers_string = re.sub("[()]", ",", workers_string)

    roles = ["produce", "engineer", "direct", "music", "supervis", "master", "mix", 
             "compose", "artist", "conduct", "arrange", "songwrite", "compilation", "restoration", "write", "solo", 
             "video", "album", "notes", "lyricist"]

    for i in range(0, len(roles)):
        workers_string = re.sub(f"({roles[i]}\w*)", "", workers_string)

    workers_string = re.sub("[^a-zA-Z0-9,.\+\-\*'$äöüÄÖÜßáéíóúüñ]", " ", workers_string)
    workers = workers_string.split(",")

    for worker in workers:  
        worker = re.sub(",", "", worker).strip()

    return workers

def clean_grammy_artists(artists_string):
    """Clean the artist column of the grammy awards dataframe and return a list of names."""
    artists_string = re.sub("&", ",", artists_string)
    artists_string = re.sub("(\weaturing)", ",", artists_string)
    artists = artists_string.split(",")
    return artists


In [6]:
### Clean OMDb movie data.

# Load data.
omdb_movie_data_list = load_data("omdb_movie_data_list.json")

# Extract RottenTomatoes rating from nested Ratings dictionary; add rtRating key and value to dictionary. 
for movie in omdb_movie_data_list:
    movie["rtRating"] = get_rt_score(movie)

# Load movie data to a data frame and set the index to imdbid.
omdb_df = pd.DataFrame(omdb_movie_data_list)
omdb_df.set_index("imdbID", inplace=True)
display(omdb_df.head())

# Drop irrelevant columns.
omdb_df.drop(labels=["Poster", "Awards", "Website", "Ratings", "Response", "Error"], axis=1, inplace=True)

# Replace all "N/A" values with NaN.
omdb_df.replace(to_replace="N/A", value=np.nan, inplace=True)

# Count NaN values in each column. 
omdb_df.isna().sum()

# Drop all rows where Type is not "movie".
display(omdb_df["Type"].value_counts())
omdb_df.dropna(subset=["Type"], inplace=True)

# Remove all rows where Title, imdbRating, Genre and Runtime are NaN.
omdb_df.dropna(subset=["Title", "imdbRating", "Genre", "Runtime"], inplace=True)

# Count NaN values in each column again. 
display(omdb_df.isna().sum())

# Clean Writers column.
omdb_df["Writer"] = omdb_df["Writer"].astype("str").map(lambda x: clean_writers(x))

# Clean Runtime column.
omdb_df["Runtime"] = omdb_df["Runtime"].astype("str").map(lambda x: clean_runtime(x))

# Clean imdbVotes column.
omdb_df["imdbVotes"] = omdb_df["imdbVotes"].astype("str").map(lambda x: clean_imdbvotes(x))

# Clean BoxOffice column.
omdb_df["BoxOffice"] = omdb_df["BoxOffice"].astype("str").map(lambda x: clean_boxoffice(x))

# Split the long strings ("ls") in each column into lists.
ls_cols = ["Genre", "Country", "Writer", "Actors", "Production", "Language"]
for col in ls_cols:
    omdb_df[col] = omdb_df[col].map(lambda x: split_long_string(x), na_action='ignore')

# Clean Country column.
omdb_df["Country"] = omdb_df["Country"].map(lambda x: clean_country(x), na_action='ignore') 

# Convert dtypes.
omdb_df["Released"] = pd.to_datetime(omdb_df["Released"])

display(omdb_df.dtypes)
display(omdb_df.head())

,Response,Error,rtRating,Title,Year,Rated,Released,Runtime,Genre,Director,...,Poster,Ratings,Metascore,imdbRating,imdbVotes,Type,DVD,BoxOffice,Production,Website
imdbID,,,,,,,,,,,,,,,,,,,,,
NaN,False,Movie not found!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt1737714,True,NaN,NaN,Salvation Road,2010,N/A,N/A,16 min,"Short, Drama",Ka'ramuu Kush,...,https://m.media-amazon.com/images/M/MV5BMjg3Mz...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.8,22,movie,N/A,N/A,N/A,N/A
tt0758786,True,NaN,73%,Stuck,2007,R,09 Jan 2009,85 min,"Comedy, Crime, Drama",Stuart Gordon,...,https://m.media-amazon.com/images/M/MV5BNTdkNT...,"[{'Source': 'Internet Movie Database', 'Value'...",61,6.4,"9,624",movie,28 Jan 2017,"$67,505","Amicus Productions, Tumidor, Prodigy Pictures",N/A
tt0161003,True,NaN,NaN,Train Ride,2000,R,N/A,93 min,Drama,Rel Dowdell,...,https://m.media-amazon.com/images/M/MV5BMTk2ND...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,4.1,233,movie,15 Mar 2005,N/A,N/A,N/A
tt5580266,True,NaN,97%,The Hate U Give,2018,PG-13,19 Oct 2018,133 min,"Crime, Drama",George Tillman Jr.,...,https://m.media-amazon.com/images/M/MV5BZDVkMW...,"[{'Source': 'Internet Movie Database', 'Value'...",81,7.5,"32,803",movie,08 Jan 2019,"$29,719,483","Fox 2000 Pictures, Temple Hill Entertainment, ...",N/A


movie    25148
Name: Type, dtype: int64

rtRating       6327
Title             0
Year              0
Rated          3859
Released        748
Runtime           0
Genre             0
Director        317
Writer         1133
Actors           69
Plot            380
Language        231
Country          80
Metascore     13230
imdbRating        0
imdbVotes         0
Type              0
DVD            6980
BoxOffice     13073
Production     8335
dtype: int64

rtRating              object
Title                 object
Year                  object
Rated                 object
Released      datetime64[ns]
Runtime               object
Genre                 object
Director              object
Writer                object
Actors                object
Plot                  object
Language              object
Country               object
Metascore             object
imdbRating            object
imdbVotes             object
Type                  object
DVD                   object
BoxOffice             object
Production            object
dtype: object

,rtRating,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Metascore,imdbRating,imdbVotes,Type,DVD,BoxOffice,Production
imdbID,,,,,,,,,,,,,,,,,,,,
tt1737714,NaN,Salvation Road,2010,NaN,NaT,16,"[Short, Drama]",Ka'ramuu Kush,"[Michael Cobian, Michael Cobian, Greg Hendrick...","[Russell Hornsby, Roger Guenveur Smith, Michae...",Business becomes unusual for an unsuspecting h...,[English],[USA],NaN,6.8,22,movie,NaN,nan,NaN
tt0758786,73%,Stuck,2007,R,2009-01-09,85,"[Comedy, Crime, Drama]",Stuart Gordon,"[John Strysik, Stuart Gordon]","[Mena Suvari, Stephen Rea, Russell Hornsby]","Brandi is a hard-partying, overworked, nursing...","[English, Spanish]","[Canada, United Kingdom, USA, Germany]",61,6.4,9624,movie,28 Jan 2017,67505,"[Amicus Productions, Tumidor, Prodigy Pictures]"
tt0161003,NaN,Train Ride,2000,R,NaT,93,[Drama],Rel Dowdell,[Rel Dowdell],"[Dakota Anderson, Thomas Braxton Jr., Joe Clai...",Three college seniors who are thinking about n...,[English],[USA],NaN,4.1,233,movie,15 Mar 2005,nan,NaN
tt5580266,97%,The Hate U Give,2018,PG-13,2018-10-19,133,"[Crime, Drama]",George Tillman Jr.,"[Audrey Wells, Angie Thomas]","[Amandla Stenberg, Regina Hall, Russell Hornsby]",Starr Carter is constantly switching between t...,[English],[USA],81,7.5,32803,movie,08 Jan 2019,29719483,"[Fox 2000 Pictures, Temple Hill Entertainment,..."
tt0444631,60%,Forgiven,2006,Not Rated,2006-01-23,81,[Drama],Paul Fitzgerald,[Paul Fitzgerald],"[Paul Fitzgerald, Russell Hornsby, Kate Jennin...",On the eve of his campaign launch for a seat i...,[English],[USA],NaN,6.4,118,movie,11 Nov 2016,nan,NaN


In [ ]:
### Clean OMDb movie data 2.

omdb_movie_data_list_2 = load_data("omdb_movie_data_list_2.json")

# Extract RottenTomatoes rating from nested Ratings dictionary; add rtRating key and value to dictionary. 
for movie in omdb_movie_data_list_2:
    movie["rtRating"] = get_rt_score(movie)

# Load movie data to a data frame and set the index to imdbid.
omdb_df_2 = pd.DataFrame(omdb_movie_data_list_2)
omdb_df_2.set_index("imdbID", inplace=True)
display(omdb_df_2.head())

# Drop irrelevant columns.
omdb_df_2.drop(labels=["Poster", "Awards", "Website", "Ratings", "Response", "Error"], axis=1, inplace=True)

# Replace all "N/A" values with NaN.
omdb_df_2.replace(to_replace="N/A", value=np.nan, inplace=True)

# Count NaN values in each column. 
omdb_df_2.isna().sum()

# Drop all rows where Type is not "movie".
display(omdb_df["Type"].value_counts())
omdb_df_2.dropna(subset=["Type"], inplace=True)

# Remove all rows where Title, imdbRating, Genre and Runtime are NaN.
omdb_df_2.dropna(subset=["Title", "imdbRating", "Genre", "Runtime"], inplace=True)

# Count NaN values in each column again. 
display(omdb_df_2.isna().sum())

# Clean Writers column.
omdb_df_2["Writer"] = omdb_df_2["Writer"].astype("str").map(lambda x: clean_writers(x))

# Clean Runtime column.
omdb_df_2["Runtime"] = omdb_df_2["Runtime"].astype("str").map(lambda x: clean_runtime(x))

# Clean imdbVotes column.
omdb_df_2["imdbVotes"] = omdb_df_2["imdbVotes"].astype("str").map(lambda x: clean_imdbvotes(x))

# Clean BoxOffice column.
omdb_df_2["BoxOffice"] = omdb_df_2["BoxOffice"].astype("str").map(lambda x: clean_boxoffice(x))

# Split the long strings ("ls") in each column into lists.
ls_cols = ["Genre", "Country", "Writer", "Actors", "Production", "Language"]
for col in ls_cols:
    omdb_df_2[col] = omdb_df_2[col].map(lambda x: split_long_string(x), na_action='ignore')

# Clean Country column.
omdb_df_2["Country"] = omdb_df_2["Country"].map(lambda x: clean_country(x), na_action='ignore') 

# Convert dtypes.
omdb_df_2["Released"] = pd.to_datetime(omdb_df_2["Released"])

display(omdb_df_2.dtypes)
display(omdb_df_2.head())

In [6]:
### Clean TMDb movie data.

# Load data.
tmdb_movie_data_list = load_data("tmdb_movie_data_list.json")

# Add relevant key-value pairs to movie_data dictionaries, load into dataframes and store in a list.
movie_dataframes = []
for movie_data in tmdb_movie_data_list:
    movie_data = append_release_dates(append_keywords(movie_data))
    movie_df = pd.DataFrame.from_dict(movie_data, orient="index")
    movie_df = movie_df.transpose()
    movie_dataframes.append(movie_df)

# Concatenate the dataframes.
tmdb_df = pd.concat(movie_dataframes, axis=0, ignore_index=True)

# Check dictonary and dataframe lengths.
assert len(tmdb_movie_data_list) == len(tmdb_df)

# Convert release date columns to datetime objects.
tmdb_df.iloc[:, 6:] = tmdb_df.iloc[:, 6:].apply(pd.to_datetime)
display(tmdb_df.dtypes)

# Drop nested dictionary columns (Release_Dates, Keywords). 
tmdb_df.drop(labels=["Release_Dates", "Keywords"], axis=1, inplace=True)
display(tmdb_df.head())
display(tmdb_df.info())

IMDb_ID                            object
TMDb_ID                            object
Alternative_Titles                 object
Release_Dates                      object
Keywords                           object
                             ...         
MP_release_date       datetime64[ns, UTC]
AS_release_date       datetime64[ns, UTC]
NA_release_date       datetime64[ns, UTC]
DM_release_date       datetime64[ns, UTC]
VI_release_date       datetime64[ns, UTC]
Length: 142, dtype: object

,IMDb_ID,TMDb_ID,Alternative_Titles,Keyword_List,US_release_date,PL_release_date,AU_release_date,TR_release_date,GB_release_date,IT_release_date,...,PG_release_date,ME_release_date,SR_release_date,KP_release_date,MU_release_date,MP_release_date,AS_release_date,NA_release_date,DM_release_date,VI_release_date
0,tt1846717,189911,[],[],2011-03-31 00:00:00+00:00,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,tt1737714,132297,[],[],2010-01-01 00:00:00+00:00,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,tt0758786,13848,"[Em Rota de Colisão, Стопор]","[taxi, garage, nightclub, hit-and-run, party, ...",2007-05-21 00:00:00+00:00,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,tt0161003,246651,[],[],2005-02-17 00:00:00+00:00,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,tt5580266,470044,"[The Hate U Give - La haine qu'on donne, The H...","[based on novel or book, riot, trauma, protest...",2018-10-19 00:00:00+00:00,2019-01-25 00:00:00+00:00,2018-11-12 00:00:00+00:00,2019-02-22 00:00:00+00:00,2018-10-22 00:00:00+00:00,2019-03-14 00:00:00+00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Columns: 140 entries, IMDb_ID to VI_release_date
dtypes: datetime64[ns, UTC](136), object(4)
memory usage: 2.4+ MB


None

In [5]:
### Clean TMDb movie data 2.

# Load data.
tmdb_movie_data_list_2 = load_data("tmdb_movie_data_list_2.json")

# Add relevant key-value pairs to movie_data dictionaries, load into dataframes and store in a list.
movie_dataframes_2 = []
for movie_data in tmdb_movie_data_list_2:
    movie_data = append_release_dates(append_keywords(movie_data))
    movie_df = pd.DataFrame.from_dict(movie_data, orient="index")
    movie_df = movie_df.transpose()
    movie_dataframes_2.append(movie_df)

# Concatenate the dataframes.
tmdb_df_2 = pd.concat(movie_dataframes_2, axis=0, ignore_index=True)

# Check dictonary and dataframe lengths.
assert len(tmdb_movie_data_list_2) == len(tmdb_df_2)

# Convert release date columns to datetime objects.
tmdb_df_2.iloc[:, 6:] = tmdb_df_2.iloc[:, 6:].apply(pd.to_datetime)
display(tmdb_df_2.dtypes)

# Drop nested dictionary columns (Release_Dates, Keywords). 
tmdb_df_2.drop(labels=["Release_Dates", "Keywords"], axis=1, inplace=True)
display(tmdb_df_2.head())
display(tmdb_df_2.info())


IMDb_ID                            object
TMDb_ID                            object
Alternative_Titles                 object
Release_Dates                      object
Keywords                           object
                             ...         
XG_release_date       datetime64[ns, UTC]
MD_release_date       datetime64[ns, UTC]
LR_release_date       datetime64[ns, UTC]
IO_release_date       datetime64[ns, UTC]
XK_release_date       datetime64[ns, UTC]
Length: 161, dtype: object

,IMDb_ID,TMDb_ID,Alternative_Titles,Keyword_List,US_release_date,GB_release_date,NO_release_date,SE_release_date,IL_release_date,IT_release_date,...,MZ_release_date,TO_release_date,TG_release_date,MG_release_date,GY_release_date,XG_release_date,MD_release_date,LR_release_date,IO_release_date,XK_release_date
0,tt1773432,78243,[The Legend of Jake Kincaid],"[redemption, cowboy]",2011-03-22 00:00:00+00:00,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,None,802917,[],[],NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,tt6149818,432987,[],"[philadelphia, pennsylvania, hip-hop, family l...",NaT,2017-08-18 00:00:00+00:00,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,tt7637160,749256,[],[],NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,None,None,[],[],NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12110 entries, 0 to 12109
Columns: 159 entries, IMDb_ID to XK_release_date
dtypes: datetime64[ns, UTC](155), object(4)
memory usage: 14.7+ MB


None

In [ ]:
# Clean cast and crew data.

# Load data.
cast_crew_data_list = load_data("cast_crew_data_list.json")

# Flatten cast and crew data into dataframes and append dataframes to lists.
cast_dfs = []
crew_dfs = []
for cc_data in cast_crew_data_list:
    cast_df = pd.json_normalize(cc_data, record_path="Cast", meta=["TMDb_ID"])
    cast_dfs.append(cast_df[:15]) # only append the first 15 actors (first billed cast)
    crew_df = pd.json_normalize(cc_data, record_path="Crew", meta=["TMDb_ID"])
    crew_dfs.append(crew_df)

# Concatenate the dataframes. 
cast_df = pd.concat(cast_dfs)
crew_df = pd.concat(crew_dfs)

# Set indexes.
cast_df.set_index("TMDb_ID", inplace=True)
crew_df.set_index("TMDb_ID", inplace=True)

# Drop irrelevant columns.
cast_df.drop(labels=["adult", "popularity", "profile_path", "cast_id", "character", "credit_id"], axis=1, inplace=True)
crew_df.drop(labels=["adult", "popularity", "profile_path", "known_for_department", "credit_id"], axis=1, inplace=True)

# Check column dtypes.
display(cast_df.dtypes)
display(crew_df.dtypes)

# Display the heads.
display(cast_df.head())
display(crew_df.head())


gender                   int64
id                       int64
known_for_department    object
name                    object
original_name           object
order                    int64
dtype: object

gender           float64
id               float64
name              object
original_name     object
department        object
job               object
dtype: object

,gender,id,known_for_department,name,original_name,order
TMDb_ID,,,,,,
189911,2,62649,Acting,Russell Hornsby,Russell Hornsby,0
189911,0,563413,Acting,Paola Mendoza,Paola Mendoza,1
189911,2,10883,Acting,Rick Yune,Rick Yune,2
189911,0,1922769,Acting,Avelina Salazar,Avelina Salazar,3
189911,0,1922770,Acting,Jesse Dubus,Jesse Dubus,4


,gender,id,name,original_name,department,job
TMDb_ID,,,,,,
189911,2.0,92784.0,Barry Jenkins,Barry Jenkins,Directing,Director
189911,2.0,92784.0,Barry Jenkins,Barry Jenkins,Production,Producer
189911,2.0,92784.0,Barry Jenkins,Barry Jenkins,Writing,Writer
189911,2.0,960350.0,James Laxton,James Laxton,Crew,Cinematography
189911,2.0,983343.0,Keegan DeWitt,Keegan DeWitt,Sound,Music


In [6]:
# Clean cast and crew data 2.

# Load data.
cast_crew_data_list_2 = load_data("cast_crew_data_list_2.json")

# Flatten cast and crew data into dataframes and append dataframes to lists.
cast_dfs = []
crew_dfs = []
for cc_data in cast_crew_data_list_2:
    cast_df = pd.json_normalize(cc_data, record_path="Cast", meta=["TMDb_ID"])
    cast_dfs.append(cast_df[:15]) # only append the first 15 actors (first billed cast)
    crew_df = pd.json_normalize(cc_data, record_path="Crew", meta=["TMDb_ID"])
    crew_dfs.append(crew_df)

# Concatenate the dataframes. 
cast_df_2 = pd.concat(cast_dfs)
crew_df_2 = pd.concat(crew_dfs)

# Set indexes.
cast_df_2.set_index("TMDb_ID", inplace=True)
crew_df_2.set_index("TMDb_ID", inplace=True)

# Drop irrelevant columns.
cast_df_2.drop(labels=["adult", "popularity", "profile_path", "cast_id", "character", "credit_id"], axis=1, inplace=True)
crew_df_2.drop(labels=["adult", "popularity", "profile_path", "known_for_department", "credit_id"], axis=1, inplace=True)

# Check column dtypes.
display(cast_df_2.dtypes)
display(crew_df_2.dtypes)

# Display the heads.
display(cast_df_2.head())
display(crew_df_2.head())

gender                  float64
id                      float64
known_for_department     object
name                     object
original_name            object
order                   float64
dtype: object

gender           float64
id               float64
name              object
original_name     object
department        object
job               object
dtype: object

,gender,id,known_for_department,name,original_name,order
TMDb_ID,,,,,,
78243,2.0,75022.0,Acting,Alan Autry,Alan Autry,0.0
78243,0.0,938537.0,Acting,Kimberlee Autry,Kimberlee Autry,1.0
78243,0.0,938538.0,Acting,Lauren Autry,Lauren Autry,2.0
78243,2.0,106119.0,Acting,David Hart,David Hart,3.0
78243,0.0,939333.0,Acting,Jim Tuck,Jim Tuck,4.0


,gender,id,name,original_name,department,job
TMDb_ID,,,,,,
78243,2.0,75022.0,Alan Autry,Alan Autry,Directing,Director
78243,2.0,75022.0,Alan Autry,Alan Autry,Writing,Writer
78243,0.0,938536.0,Greg Grey,Greg Grey,Writing,Writer
802917,2.0,46915.0,Neil Elman,Neil Elman,Production,Executive Producer
802917,1.0,59426.0,Jennifer Rudnicke,Jennifer Rudnicke,Production,Casting


In [8]:
### Clean actor data.

# Load the data.
actor_data_list = load_data("actor_data_list.json")

# Load to dataframe.
actor_df = pd.DataFrame(actor_data_list)
display(actor_df.head())
display(actor_df.isna().sum())

# Convert column dtypes.
actor_df["TMDb_ID"] = actor_df["TMDb_ID"].astype("str")
actor_df["Birthday"] = pd.to_datetime(actor_df["Birthday"])
display(actor_df.dtypes)
display(actor_df.head())


,IMDb_ID,TMDb_ID,Gender,Birthday,Movie_Credits
0,nm0395203,62649,2,1974-05-15,"[Remigration, Salvation Road, Stuck, Train Rid..."
1,nm0693446,167566,0,None,"[37, Rounding, Ma Rainey's Black Bottom, Let M..."
2,nm2008303,1316498,2,1970-04-01,"[Goldie, Greta, First Match, Skate Kitchen, Jo..."
3,nm5826571,2659492,2,None,[]
4,nm1844359,1694522,2,None,"[Road Trip: Beer Pong, What Men Want]"


IMDb_ID            0
TMDb_ID            0
Gender             0
Birthday         560
Movie_Credits      0
dtype: int64

IMDb_ID                  object
TMDb_ID                  object
Gender                    int64
Birthday         datetime64[ns]
Movie_Credits            object
dtype: object

,IMDb_ID,TMDb_ID,Gender,Birthday,Movie_Credits
0,nm0395203,62649,2,1974-05-15,"[Remigration, Salvation Road, Stuck, Train Rid..."
1,nm0693446,167566,0,NaT,"[37, Rounding, Ma Rainey's Black Bottom, Let M..."
2,nm2008303,1316498,2,1970-04-01,"[Goldie, Greta, First Match, Skate Kitchen, Jo..."
3,nm5826571,2659492,2,NaT,[]
4,nm1844359,1694522,2,NaT,"[Road Trip: Beer Pong, What Men Want]"


In [9]:
### Clean soundtrack credits data (stc).

# Load the data.
soundtrack_credits_data_list = load_data("soundtrack_credits_data_list.json")

# Load into pandas dataframe.
soundtrack_df = pd.DataFrame(soundtrack_credits_data_list)
soundtrack_df.set_index("IMDb_ID", inplace=True)
display(soundtrack_df.dtypes)
display(soundtrack_df.head())


Soundtrack_Credits    object
dtype: object

,Soundtrack_Credits
IMDb_ID,
tt0265298,"[[nm1028472, Greg Camp], [nm1704901, Smash Mou..."
tt0758786,"[[updates/guide/soundtracks, Soundtracks submi..."
tt2671706,"[[nm0177492, Sam Cooke], [nm0205626, Viola Dav..."
tt5580266,"[[nm1159305, Jadakiss], [nm1159305, Jadakiss],..."
tt1846717,"[[updates/guide/soundtracks, Soundtracks submi..."


In [11]:
### Clean soundtrack credits data (stc) 2.

# Load the data.
soundtrack_credits_data_list_2 = load_data("soundtrack_credits_data_list_2.json")

# Load into pandas dataframe.
soundtrack_df_2 = pd.DataFrame(soundtrack_credits_data_list_2)
soundtrack_df_2.set_index("IMDb_ID", inplace=True)
display(soundtrack_df_2.dtypes)
display(soundtrack_df_2.head())

Soundtrack_Credits    object
dtype: object

,Soundtrack_Credits
IMDb_ID,
tt0264464,"[[nm0022750, Paul Alter], [nm3097133, Robert I..."
tt1572315,"[[nm2931310, Preston Hart], [nm4578829, Milad ..."
tt7637160,"[[updates/guide/soundtracks, Soundtracks submi..."
tt3094914,"[[nm3534649, Jungee Marcelo], [nm4800418, Zia ..."
tt1773432,"[[nm5218879, Floyd VanLaningham]]"


In [10]:
### Clean the Golden Globe, Oscar and Grammy awards datasets.

# Load Golden Globe data to dataframe.
gg_awards_df = pd.read_csv("golden_globe_awards.csv")

# Normalise year_award values and convert to datetime object.
gg_awards_df["year_award"] = gg_awards_df["year_award"].astype("str").map(lambda x: convert_award_date(x))

# Drop irrelevant columns.
display(gg_awards_df["win"].value_counts())
gg_awards_df.drop(labels=["year_film", "ceremony", "category", "film", "win"], axis=1, inplace=True)
display(gg_awards_df.head())
display(gg_awards_df.dtypes)

# Load Grammy data to dataframe.
grammy_awards_df = pd.read_csv("the_grammy_awards.csv")

# Drop irrelevant columns.
display(grammy_awards_df["winner"].value_counts())
grammy_awards_df.drop(labels=["year", "title", "updated_at", "category", "nominee", "img", "winner"], axis=1, inplace=True)

# Convert published_at date to datetime and remove timestamps.
grammy_awards_df["published_at"] = grammy_awards_df["published_at"].map(lambda x: pd.to_datetime(x).date())
grammy_awards_df["published_at"] = pd.to_datetime(grammy_awards_df["published_at"])

# Clean "artist" and "workers" column values.
grammy_awards_df["artist"] = grammy_awards_df["artist"].astype("str").map(lambda x: clean_grammy_artists(x))
grammy_awards_df["workers"] = grammy_awards_df["workers"].astype("str").map(lambda x: clean_grammy_workers(x))
display(grammy_awards_df.dtypes)
display(grammy_awards_df.head())

# Load Oscars data to dataframe.
oscar_awards_df = pd.read_csv("the_oscar_award.csv")

# Normalise year_ceremony values and convert to datetime object.
oscar_awards_df["year_ceremony"] = oscar_awards_df["year_ceremony"].astype("str").map(lambda x: convert_award_date(x))
display(oscar_awards_df.dtypes)

# Drop all rows where the nominee/s did not win.
oscar_awards_df["winner"].replace(to_replace=False, value=np.nan, inplace=True)
oscar_awards_df.dropna(subset=["winner"], inplace=True)

# Drop irrelevant columns.
oscar_awards_df.drop(labels=["year_film", "ceremony", "category", "film", "winner"], axis=1, inplace=True)
display(oscar_awards_df.head())


False    6166
True     1825
Name: win, dtype: int64

,year_award,nominee
0,1944-04-01,Katina Paxinou
1,1944-04-01,Akim Tamiroff
2,1944-04-01,Henry King
3,1944-04-01,The Song Of Bernadette
4,1944-04-01,Jennifer Jones


year_award    datetime64[ns]
nominee               object
dtype: object

True    4810
Name: winner, dtype: int64

published_at    datetime64[ns]
artist                  object
workers                 object
dtype: object

,published_at,artist,workers
0,2020-05-19,[Billie Eilish],"[Finneas O'Connell, Rob Kinelski , Finneas O..."
1,2020-05-19,[Bon Iver],"[BJ Burton, Brad Cook, Chris Messina , Just..."
2,2020-05-19,[Ariana Grande],"[Charles Anderson, Tommy Brown, Michael Fost..."
3,2020-05-19,[H.E.R.],"[Rodney Darkchild Jerkins, Joseph Hurtado, ..."
4,2020-05-19,[Khalid],"[Disclosure , Denis Kosiak, Ingmar Carlson, ..."


year_film                 int64
year_ceremony    datetime64[ns]
ceremony                  int64
category                 object
name                     object
film                     object
winner                     bool
dtype: object

,year_ceremony,name
1,1928-04-01,Emil Jannings
3,1928-04-01,Janet Gaynor
6,1928-04-01,William Cameron Menzies
9,1928-04-01,Charles Rosher
10,1928-04-01,Karl Struss


In [11]:
### Clean Box Office data.

# Load data.
box_office_data_list = load_data("box_office_data_list.json")
boxoffice_df = pd.DataFrame(box_office_data_list)
boxoffice_df.set_index("IMDb_ID", inplace=True)

# Clean gross values. 
boxoffice_df["Opening_Weekend_Gross"] = boxoffice_df["Opening_Weekend_Gross"].map(lambda x: clean_boxoffice(x))
boxoffice_df["Worldwide_Gross"] = boxoffice_df["Worldwide_Gross"].map(lambda x: clean_boxoffice(x))
display(len(boxoffice_df))

# Remove rows with no value in both columns.
boxoffice_df.replace(to_replace="", value=np.nan, inplace=True)
boxoffice_df.dropna(subset=["Opening_Weekend_Gross", "Worldwide_Gross"], how='all', inplace=True)
display(len(boxoffice_df))
display(boxoffice_df.head())

1943

978

,Opening_Weekend_Gross,Worldwide_Gross
IMDb_ID,,
tt0758786,8844,151449
tt5580266,512035,34934009
tt0265298,11554015,52970014
tt0367479,11500000,62657868
tt2671706,129462,64414761


In [7]:
### Clean Box Office data 2.

# Load data.
box_office_data_list_2 = load_data("box_office_data_list_2.json")
boxoffice_df_2 = pd.DataFrame(box_office_data_list_2)
boxoffice_df_2.set_index("IMDb_ID", inplace=True)

# Clean gross values. 
boxoffice_df_2["Opening_Weekend_Gross"] = boxoffice_df_2["Opening_Weekend_Gross"].map(lambda x: clean_boxoffice(x))
boxoffice_df_2["Worldwide_Gross"] = boxoffice_df_2["Worldwide_Gross"].map(lambda x: clean_boxoffice(x))
display(len(boxoffice_df_2))

# Remove rows with no value in both columns.
boxoffice_df_2.replace(to_replace="", value=np.nan, inplace=True)
boxoffice_df_2.dropna(subset=["Opening_Weekend_Gross", "Worldwide_Gross"], how='all', inplace=True)
display(len(boxoffice_df_2))
display(boxoffice_df_2.head())

11140

4163

,Opening_Weekend_Gross,Worldwide_Gross
IMDb_ID,,
tt6149818,6591,50022
tt0264464,30053627,352114312
tt3094914,None,2352001
tt1572315,21744470,47340586
tt0455841,473062,1103037


In [12]:
### Load remaining datasets to dataframes.

# Budget data.
budget_data_list = load_data("budget_data_list.json")
budget_df = pd.DataFrame(budget_data_list)
budget_df.set_index("TMDb_ID", inplace=True)

# Name & ID data (collected in the initial IMDb scrape). 
name_id_list = load_data("name_id_list.json")
name_id_df = pd.DataFrame(name_id_list)

# Actors with no TMDb profiles/data.
actor_not_found_list = load_data("actor_not_found_list.json")
actor_na_df = pd.DataFrame(actor_not_found_list)

In [8]:
# Budget data.
budget_data_list_2 = load_data("budget_data_list_2.json")
budget_df_2 = pd.DataFrame(budget_data_list_2)
budget_df_2.set_index("TMDb_ID", inplace=True)

In [9]:
### Connect and load data to PostgreSQL Database.
import psycopg2
import hidden
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

# Return an SQLAlchemy connection string.
secrets = hidden.secrets()
conn_string = hidden.alchemy(secrets)

# Create an engine. 
engine = create_engine(conn_string)

In [13]:
# Write records stored in DataFrames to SQL databases.
with engine.begin() as connection:
    omdb_df.to_sql("omdb_df_data", con=connection, if_exists="replace") #1
    omdb_df_2.to_sql("omdb_df_data_2", con=connection, if_exists="replace") #2
    tmdb_df.to_sql("tmdb_df_data", con=connection, if_exists="replace") #1
    tmdb_df_2.to_sql("tmdb_df_data_2", con=connection, if_exists="replace") #2
    cast_df.to_sql("cast_df_data", con=connection, if_exists="replace") #1
    cast_df_2.to_sql("cast_df_data_2", con=connection, if_exists="replace") #2
    crew_df.to_sql("crew_df_data", con=connection, if_exists="replace") #1
    crew_df_2.to_sql("crew_df_data_2", con=connection, if_exists="replace") #2
    budget_df.to_sql("budget_df_data", con=connection, if_exists="replace") #1
    budget_df_2.to_sql("budget_df_data_2", con=connection, if_exists="replace") #2
    boxoffice_df.to_sql("boxoffice_df_data", con=connection, if_exists="replace") #1
    boxoffice_df_2.to_sql("boxoffice_df_data_2", con=connection, if_exists="replace") #2
    soundtrack_df.to_sql("soundtrack_df_data", con=connection, if_exists="replace") #1
    soundtrack_df_2.to_sql("soundtrack_df_data_2", con=connection, if_exists="replace") #2
    grammy_awards_df.to_sql("grammy_awards_df_data", con=connection, if_exists="replace") #1
    gg_awards_df.to_sql("gg_awards_df_data", con=connection, if_exists="replace") #1
    oscar_awards_df.to_sql("oscar_awards_df_data", con=connection, if_exists="replace") #1
    

In [ ]:
### PostgreSQL statements to alter and merge tables (executed in DBeaver) #1 
"""DELETE FROM soundtrack_df_data
WHERE "Soundtrack_Credits" LIKE "%update%"
OR "Soundtrack_Credits" LIKE "{}";

ALTER TABLE gg_awards_df_data 
RENAME COLUMN nominee TO winner;

ALTER TABLE oscar_awards_df_data 
RENAME COLUMN year_ceremony TO year_award;

ALTER TABLE oscar_awards_df_data 
RENAME COLUMN name TO winner;

ALTER TABLE "the_grammy_awards.csv" 
RENAME TO grammy_awards_df_data;

ALTER TABLE grammy_awards_df_data 
RENAME COLUMN published_at TO year_award;

ALTER TABLE grammy_awards_df_data 
ADD COLUMN winner VARCHAR(128);

ALTER TABLE tmdb_df_data 
DROP COLUMN index;

DROP TABLE IF EXISTS movie_data;

CREATE TABLE movie_data AS 
WITH 
	cte_cast_crew AS (
		SELECT 
			crewdd."TMDb_ID",
			ARRAY_AGG(crewdd.original_name) AS Movie_Cast,
			ARRAY_AGG(castdd.original_name) AS Movie_Crew
		FROM crew_df_data AS crewdd
		JOIN cast_df_data AS castdd
		 ON crewdd."TMDb_ID" = castdd."TMDb_ID"
		GROUP BY crewdd."TMDb_ID" 
),
	cte_merged_data AS (
		SELECT DISTINCT 
			odd.*,
			tdd."US_release_date", 
			tdd."GB_release_date",
			tdd."Alternative_Titles",
			tdd."Keyword_List", 
			ccc.Movie_Cast,
			ccc.Movie_Crew,
			bdd."Budget",
			bdd2."Opening_Weekend_Gross",
			bdd2."Worldwide_Gross",
			sdd."Soundtrack_Credits"
 		FROM omdb_df_data AS odd
 		LEFT JOIN tmdb_df_data AS tdd
 		 ON tdd."IMDb_ID" = odd."imdbID" 
 		LEFT JOIN cte_cast_crew AS ccc 
 		 ON ccc."TMDb_ID" = tdd."TMDb_ID"
 		LEFT JOIN budget_df_data AS bdd
 		 ON bdd."TMDb_ID" = tdd."TMDb_ID"
  		LEFT JOIN soundtrack_df_data AS sdd
  		 ON sdd."IMDb_ID" = odd."imdbID"
  		LEFT JOIN boxoffice_df_data AS bdd2
 		 ON odd."imdbID" = bdd2."IMDb_ID" 
 )			
(SELECT
	cterd.*,	
	CARDINALITY(cterd."Alternative_Titles"::TEXT[]) AS Alternative_Titles
 FROM cte_merged_data AS cterd);

ALTER TABLE movie_data 
DROP COLUMN "Alternative_Titles",
DROP COLUMN "Type",
ALTER COLUMN "Runtime" SET DATA TYPE INT USING "Runtime"::INT,
ALTER COLUMN "Genre" SET DATA TYPE TEXT[] USING "Genre"::TEXT[],
ALTER COLUMN "Writer" SET DATA TYPE TEXT[] USING "Writer"::TEXT[],
ALTER COLUMN "Actors" SET DATA TYPE TEXT[] USING "Actors"::TEXT[],
ALTER COLUMN "Production" SET DATA TYPE TEXT[] USING "Production"::TEXT[],
ALTER COLUMN "Language" SET DATA TYPE TEXT[] USING "Language"::TEXT[],
ALTER COLUMN "Country" SET DATA TYPE TEXT[] USING "Country"::TEXT[],
ALTER COLUMN "imdbVotes" SET DATA TYPE INT USING "imdbVotes"::INT,
ALTER COLUMN "imdbRating" SET DATA TYPE NUMERIC USING "imdbRating"::NUMERIC,
ALTER COLUMN "Keyword_List" SET DATA TYPE TEXT[] USING "Keyword_List"::TEXT[],
ALTER COLUMN "Opening_Weekend_Gross" SET DATA TYPE NUMERIC USING "Opening_Weekend_Gross"::NUMERIC,
ALTER COLUMN "Worldwide_Gross" SET DATA TYPE NUMERIC USING "Worldwide_Gross"::NUMERIC,
ALTER COLUMN "Soundtrack_Credits" SET DATA TYPE TEXT[] USING "Soundtrack_Credits"::TEXT[];

ALTER TABLE actor_df_data 
DROP COLUMN "index";

DROP TABLE IF EXISTS actor_data;

CREATE TABLE actor_data AS
WITH cte_actor_data AS (
	SELECT
	    add2.*,
		nidd.names AS Actor
	FROM actor_df_data AS add2
	JOIN name_id_df_data AS nidd
	 ON add2."IMDb_ID" = nidd.imdb_ids
)
(SELECT DISTINCT * FROM cte_actor_data);

SELECT md.*, tdd.* 
 FROM movie_data md
 LEFT JOIN tmdb_df_data tdd
  ON md."imdbID" = tdd."IMDb_ID" 
 WHERE 
 	md."Released" IS NULL;

CREATE TABLE movie_awards_data AS
WITH 
	cte_movie_awards AS (
		SELECT 
			year_award,
	    	winner
		FROM gg_awards_df_data
		UNION 
		SELECT 
			year_award, 
			winner 
		FROM oscar_awards_df_data
)
(SELECT * FROM cte_movie_awards);

CREATE TABLE music_awards_data AS
WITH
	cte_music_awards AS (
		SELECT 
			year_award,
			(CASE WHEN (artist, workers) IS NULL 
			  THEN NULL 
		 	 ELSE CONCAT(artist, workers) 
		 	 END) winner
	    FROM 
			grammy_awards_df_data
)
(SELECT * FROM cte_music_awards);"""

In [ ]:
### PostgreSQL statements to alter and merge tables (executed in DBeaver) #2
"""DELETE FROM soundtrack_df_data_2
WHERE "Soundtrack_Credits" LIKE '%update%'
OR "Soundtrack_Credits" LIKE '{}';

ALTER TABLE tmdb_df_data_2 
DROP COLUMN index;

DROP TABLE IF EXISTS movie_data_2;

CREATE TABLE movie_data_2 AS 
WITH 
	cte_cast_crew AS (
		SELECT 
			crewdd."TMDb_ID",
			ARRAY_AGG(crewdd.original_name) AS Movie_Cast,
			ARRAY_AGG(castdd.original_name) AS Movie_Crew
		FROM crew_df_data_2 AS crewdd
		JOIN cast_df_data_2 AS castdd
		 ON crewdd."TMDb_ID" = castdd."TMDb_ID"
		GROUP BY crewdd."TMDb_ID" 
),
	cte_merged_data AS (
		SELECT DISTINCT 
			odd.*,
			tdd."US_release_date", 
			tdd."GB_release_date",
			tdd."Alternative_Titles",
			tdd."Keyword_List", 
			ccc.Movie_Cast,
			ccc.Movie_Crew,
			bdd."Budget",
			bdd2."Opening_Weekend_Gross",
			bdd2."Worldwide_Gross",
			sdd."Soundtrack_Credits"
 		FROM omdb_df_data_2 AS odd
 		LEFT JOIN tmdb_df_data_2 AS tdd
 		 ON tdd."IMDb_ID" = odd."imdbID" 
 		LEFT JOIN cte_cast_crew AS ccc 
 		 ON ccc."TMDb_ID" = tdd."TMDb_ID"
 		LEFT JOIN budget_df_data_2 AS bdd
 		 ON bdd."TMDb_ID" = tdd."TMDb_ID"
  		LEFT JOIN soundtrack_df_data_2 AS sdd
  		 ON sdd."IMDb_ID" = odd."imdbID"
  		LEFT JOIN boxoffice_df_data_2 AS bdd2
 		 ON odd."imdbID" = bdd2."IMDb_ID" 
 )			
(SELECT
	cterd.*,	
	CARDINALITY(cterd."Alternative_Titles"::TEXT[]) AS Alternative_Titles
 FROM cte_merged_data AS cterd);

ALTER TABLE movie_data_2
DROP COLUMN "Alternative_Titles",
DROP COLUMN "Type",
ALTER COLUMN "Runtime" SET DATA TYPE INT USING "Runtime"::INT,
ALTER COLUMN "Genre" SET DATA TYPE TEXT[] USING "Genre"::TEXT[],
ALTER COLUMN "Writer" SET DATA TYPE TEXT[] USING "Writer"::TEXT[],
ALTER COLUMN "Actors" SET DATA TYPE TEXT[] USING "Actors"::TEXT[],
ALTER COLUMN "Production" SET DATA TYPE TEXT[] USING "Production"::TEXT[],
ALTER COLUMN "Language" SET DATA TYPE TEXT[] USING "Language"::TEXT[],
ALTER COLUMN "Country" SET DATA TYPE TEXT[] USING "Country"::TEXT[],
ALTER COLUMN "imdbVotes" SET DATA TYPE INT USING "imdbVotes"::INT,
ALTER COLUMN "imdbRating" SET DATA TYPE NUMERIC USING "imdbRating"::NUMERIC,
ALTER COLUMN "Keyword_List" SET DATA TYPE TEXT[] USING "Keyword_List"::TEXT[],
ALTER COLUMN "Opening_Weekend_Gross" SET DATA TYPE NUMERIC USING "Opening_Weekend_Gross"::NUMERIC,
ALTER COLUMN "Worldwide_Gross" SET DATA TYPE NUMERIC USING "Worldwide_Gross"::NUMERIC,
ALTER COLUMN "Soundtrack_Credits" SET DATA TYPE TEXT[] USING "Soundtrack_Credits"::TEXT[];"""


In [ ]:
### Merge movie_data 1 and 2 (executed in Dbeaver)
"""CREATE TABLE movie_data_final AS 
WITH 
	cte_movie_data AS (
			SELECT md2.*
			FROM movie_data_2 md2
			LEFT JOIN movie_data md
				ON md2."imdbID" = md."imdbID"
) 
SELECT DISTINCT ctemd.* 
FROM cte_movie_data ctemd;"""

In [6]:
### Connect and load data to PostgreSQL Database.
import psycopg2
import hidden
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

# Return an SQLAlchemy connection string.
secrets = hidden.secrets()
conn_string = hidden.alchemy(secrets)

# Create an engine. 
engine = create_engine(conn_string)

# Load merged SQL tables to dataframes.
movie_data_df = pd.read_sql_table("movie_data", con=engine)
movie_data_df_2 = pd.read_sql_table("movie_data_2", con=engine)
actor_data_df = pd.read_sql_table("actor_data", con=engine)
movie_awards_df = pd.read_sql_table("movie_awards_data", con=engine)
music_awards_df = pd.read_sql_table("music_awards_data", con=engine)

In [7]:
### Define additional cleaning and transformation functions.

def clean_plot(plot):
    """Clean the plot text."""
    plot = re.sub("[^a-zA-Z0-9]", " ", plot).lower() # remove non alpha numeric characters
    plot = re.sub("[0-9]+", "number", plot) # convert all digits to the word "number"
    plot = [word for word in plot.split() if not word in stopwords.words()] # remove stop words
    return plot

def clean_RTrating(rtrating):
    """Clean the Rotten Tomatoes ratings."""
    rtrating = re.sub("%", "", rtrating)
    return rtrating

def clean_cast(movie_cast):
    """Remove duplicates from the movie_cast values."""
    actors = []
    for actor in movie_cast:
        if actor not in actors:
            actors.append(actor)
    return actors

def clean_crew(movie_crew):
    """Remove duplicates from the movie_crew values."""
    crew = []
    for member in movie_crew:
        if member not in crew:
            crew.append(member)
    return crew
    
def get_supporting_actors(movie_cast, lead_actors):
    """Get the supporting actors by comparing the full cast list with the lead actors list."""
    if movie_cast
    supporting_actors = set(movie_cast).difference(set(lead_actors))
    return list(supporting_actors)

In [8]:
# Merge the two movie_data dataframes. 
print(len(movie_data_df))
print(len(movie_data_df_2))

movie_data_df = pd.concat([movie_data_df, movie_data_df_2])

1800
8188


In [9]:
print(len(movie_data_df))

# Drop duplicates.
movie_data_df.drop_duplicates(subset="imdbID", inplace=True)
print(len(movie_data_df)) 

9988
9436


In [12]:
### Clean movie_data.
display(movie_data_df.head())
display(movie_data_df.dtypes)

# Fill all missing release date values.
movie_data_df["Released"].fillna(movie_data_df["US_release_date"], inplace=True)
movie_data_df["Released"].fillna(movie_data_df["GB_release_date"], inplace=True)

,imdbID,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,Keyword_List,movie_cast,movie_crew,Budget,Opening_Weekend_Gross,Worldwide_Gross,Soundtrack_Credits,alternative_titles,Type,Alternative_Titles
0,tt0056952,The Cool World,1963,None,1964-11-02 00:00:00,105,"[Crime, Drama]",Shirley Clarke,"[Shirley Clarke, Carl Lee, Warren Miller]","[Rony Clanton, Carl Lee, Yolanda Rodríguez]",...,"[jazz, gang, racism, woman director, harlem]","[Hugh A. Robertson, Robert Rossen, Carl Lee, F...","[Rony Clanton, Rony Clanton, Rony Clanton, Ron...",0.0,NaN,NaN,None,0.0,NaN,NaN
1,tt0065944,King: A Filmed Record... Montgomery to Memphis,1970,None,1970-03-24 00:00:00+00:00,185,"[Documentary, Biography, History]","Sidney Lumet, Joseph L. Mankiewicz","[Mitchell Grayson, Ely A. Landau]","[Paul Newman, Joanne Woodward, Ruby Dee, James...",...,"[civil war, alabama, racism, segregation]","[Sidney Lumet, Joseph L. Mankiewicz, Richard K...","[Martin Luther King, Martin Luther King, Marti...",0.0,NaN,NaN,None,2.0,NaN,NaN
2,tt0066559,What Do You Say to a Naked Lady?,1970,X,1970-02-18 00:00:00,85,"[Documentary, Comedy]",Allen Funt,[Allen Funt],"[Joie Addison, Laura Huston, Martin Meyers, Ka...",...,"[camera, naked, candid camera, elevator, react...","[Urs Furrer, Steve Karmen, Allen Funt, Allen F...","[Allen Funt, Allen Funt, Allen Funt, Allen Fun...",0.0,NaN,NaN,None,2.0,NaN,NaN
3,tt0067741,Shaft,1971,R,1971-07-02 00:00:00,100,"[Action, Crime, Thriller]",Gordon Parks,"[Ernest Tidyman, John D.F. Black, Ernest Tidyman]","[Richard Roundtree, Moses Gunn, Charles Cioffi...",...,"[new york city, black people, ghetto, daughter...","[Ernest Tidyman, Ernest Tidyman, Ernest Tidyma...","[Richard Roundtree, Richard Roundtree, Richard...",4000000.0,NaN,NaN,"[[nm0005002, Isaac Hayes], [nm0005002, Isaac H...",1.0,NaN,NaN
4,tt0068358,Charley-One-Eye,1973,R,1973-04-18 00:00:00+00:00,96,[Western],Don Chaffey,[Keith Leonard],"[Richard Roundtree, Roy Thinnes, Nigel Davenpo...",...,"[civil war, on the run, racism]","[James Swann, John Cameron, Don Chaffey, Tim H...","[Richard Roundtree, Richard Roundtree, Richard...",0.0,NaN,NaN,None,3.0,NaN,NaN


imdbID                                object
Title                                 object
Year                                  object
Rated                                 object
Released                              object
Runtime                               object
Genre                                 object
Director                              object
Writer                                object
Actors                                object
Plot                                  object
Language                              object
Country                               object
Metascore                             object
imdbRating                            object
imdbVotes                             object
rtRating                              object
DVD                                   object
BoxOffice                             object
Production                            object
US_release_date          datetime64[ns, UTC]
GB_release_date          datetime64[ns, UTC]
Keyword_Li

In [11]:
movie_data_df.isna().sum()

imdbID                      0
Title                       0
Year                        0
Rated                    2097
Released                  203
Runtime                     0
Genre                       0
Director                  165
Writer                      0
Actors                     31
Plot                      237
Language                  113
Country                    48
Metascore                6427
imdbRating                  0
imdbVotes                   0
rtRating                 3341
DVD                      3643
BoxOffice                   0
Production               4291
US_release_date          3644
GB_release_date          7440
Keyword_List              940
movie_cast               1118
movie_crew               1118
Budget                    940
Opening_Weekend_Gross    6699
Worldwide_Gross          6092
Soundtrack_Credits       5011
alternative_titles        940
Type                     1800
Alternative_Titles       2740
dtype: int64

In [ ]:
# Drop remaining rows without a release date. 
movie_data_df.dropna(subset=["Released"], inplace=True)
display(movie_data_df["Released"].isna().sum())

# Drop redundant date columns.
movie_data_df.drop(labels=["US_release_date", "GB_release_date"], axis=1, inplace=True)

In [ ]:
### Locate the missing values and determine if rows are redundant. 

# Remove movies without Directors information
display(movie_data_df.isna().sum())
display(movie_data_df[movie_data_df["Director"].isna()])
movie_data_df.dropna(subset=["Director"], inplace=True)

In [20]:
# Remove movies without Plot information
movie_data_df.isna().sum()
movie_data_df[movie_data_df["Plot"].isna()]
movie_data_df.dropna(subset=["Plot"], inplace=True)

In [21]:
# Remove movies without movie_cast information
movie_data_df.isna().sum()
movie_data_df[movie_data_df["movie_cast"].isna()]
movie_data_df.dropna(subset=["movie_cast"], inplace=True)

In [24]:
# Remove movies without BoxOffice information
movie_data_df.isna().sum()
movie_data_df["BoxOffice"].replace(to_replace="nan", value=np.nan, inplace=True)

# Remove movies that were for TV. 
movie_data_df = movie_data_df[~movie_data_df["Rated"].str.contains("TV", na=False)]
movie_data_df.isna().sum()

imdbID                      0
Title                       0
Year                        0
Rated                    1361
Released                    0
Runtime                     0
Genre                       0
Director                    0
Writer                      0
Actors                      6
Plot                        0
Language                   84
Country                    28
Metascore                5147
imdbRating                  0
imdbVotes                   0
rtRating                 2342
DVD                      2634
BoxOffice                   0
Production               3214
Keyword_List                0
movie_cast                  0
movie_crew                  0
Budget                      0
Opening_Weekend_Gross    5289
Worldwide_Gross          4685
Soundtrack_Credits       3651
alternative_titles          0
Type                     1696
Alternative_Titles       1696
dtype: int64

In [25]:
# Clean plot values.
movie_data_df["Plot"] = movie_data_df["Plot"].astype(str).map(lambda x: clean_plot(x))

In [26]:
# Clean Rotten Tomatoes ratings.
movie_data_df["rtRating"] = movie_data_df["rtRating"].astype(str).map(lambda x: clean_RTrating(x))

# Remove duplicates from the movie_cast.
movie_data_df["movie_cast"] = movie_data_df["movie_cast"].map(lambda x: clean_cast(x)) 

# Remove duplicates from the movie_crew.
movie_data_df["movie_crew"] = movie_data_df["movie_crew"].map(lambda x: clean_crew(x))

# Swap the movie_cast and movie_crew column names (to correct naming error in joining)
movie_data_df.rename(columns={"movie_cast": "movie_crew_temp", "movie_crew": "Movie_Cast"}, inplace=True)
movie_data_df.rename(columns={"movie_crew_temp": "Movie_Crew"}, inplace=True)

# Rename the Actors column to Lead_Actors
movie_data_df.rename(columns={"Actors": "Lead_Actors"}, inplace=True)

In [28]:
movie_data_df.isna().sum()

imdbID                      0
Title                       0
Year                        0
Rated                    1361
Released                    0
Runtime                     0
Genre                       0
Director                    0
Writer                      0
Lead_Actors                 6
Plot                        0
Language                   84
Country                    28
Metascore                5147
imdbRating                  0
imdbVotes                   0
rtRating                    0
DVD                      2634
BoxOffice                   0
Production               3214
Keyword_List                0
Movie_Crew                  0
Movie_Cast                  0
Budget                      0
Opening_Weekend_Gross    5289
Worldwide_Gross          4685
Soundtrack_Credits       3651
alternative_titles          0
Type                     1696
Alternative_Titles       1696
dtype: int64

In [29]:
movie_data_df.to_pickle("movie_data_df_temp.pkl")

In [27]:
# Create a Supporting_Actors column by splitting the movie_cast.
movie_data_df["Supporting_Actors"] = movie_data_df.apply(lambda row: get_supporting_actors(row["Movie_Cast"], row["Lead_Actors"]), axis=1) 

TypeError: 'NoneType' object is not iterable

In [ ]:
# Convert all dataframes to pickle. 
display(music_awards_df.head())
music_awards_df.to_pickle("./music_awards_df.pkl")

display(movie_awards_df.head())
movie_awards_df.to_pickle("./movie_awards_df.pkl")

display(actor_data_df.head())
actor_data_df.to_pickle("./actor_data_df.pkl")

display(movie_data_df.head())
movie_data_df.to_pickle("./movie_data_df.pkl")

,year_award,winner
0,2020-05-19,"{""Billie Eilish""}{""Finneas O'Connell"","" Rob Ki..."
1,2020-05-19,"{""Bon Iver""}{""BJ Burton"","" Brad Cook"","" Chris ..."
2,2020-05-19,"{""Ariana Grande""}{""Charles Anderson"","" Tommy B..."
3,2020-05-19,"{H.E.R.}{""Rodney Darkchild Jerkins"","" Joseph..."
4,2020-05-19,"{Khalid}{""Disclosure "","" Denis Kosiak"","" Ingma..."


,year_award,winner
0,2005-04-01,Nicollette Sheridan
1,1979-04-01,Brad Davis
2,1964-04-01,Martin Ritt
3,1963-04-01,Janet Margolin
4,2008-04-01,Grey's Anatomy


,IMDb_ID,TMDb_ID,Gender,Birthday,Movie_Credits,actor
0,nm1004267,195666,1,1976-02-08,"{Snowflakes,""Alien: Containment"",Blooded,""Shoo...",Sharon Duncan-Brewster
1,nm2351246,479206,1,1981-08-27,"{""A Stone Cold Christmas"",""Bobbi Kristina"",""Br...",Demetria McKinney
2,nm4555351,1975743,1,1989-05-11,"{""Look Again"",""The Lovely Rejects"",""Healing Me...",Lauren Neal
3,nm0376200,83976,2,1938-02-01,"{""For the Love of a Dog"",""Ghost Fever"",Clunker...",Sherman Hemsley
4,nm0940851,21356,2,1969-07-25,"{""After All"",""The Inheritance"",""The Temptation...",D.B. Woodside


,imdbID,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Lead_Actors,...,Production,Keyword_List,Movie_Crew,Movie_Cast,Budget,Opening_Weekend_Gross,Worldwide_Gross,Soundtrack_Credits,alternative_titles,Supporting_Actors
0,tt0056952,The Cool World,1963,None,1964-11-02 00:00:00,105,"[Crime, Drama]",Shirley Clarke,"[Shirley Clarke, Carl Lee, Warren Miller]","[Rony Clanton, Carl Lee, Yolanda Rodríguez]",...,None,"[jazz, gang, racism, woman director, harlem]","[Hugh A. Robertson, Robert Rossen, Carl Lee, F...","[Rony Clanton, Carl Lee, Yolanda Rodríguez, Cl...",0,NaN,NaN,None,0,"[Clarence Williams III, Richard Ward, Antonio ..."
1,tt0065944,King: A Filmed Record... Montgomery to Memphis,1970,None,1970-03-24 00:00:00+00:00,185,"[Documentary, Biography, History]","Sidney Lumet, Joseph L. Mankiewicz","[Mitchell Grayson, Ely A. Landau]","[Paul Newman, Joanne Woodward, Ruby Dee, James...",...,None,"[civil war, alabama, racism, segregation]","[Sidney Lumet, Joseph L. Mankiewicz, Richard K...","[Martin Luther King, Coretta Scott King, Ralph...",0,NaN,NaN,None,2,"[Harry Belafonte, James Baldwin, Martin Luther..."
2,tt0066559,What Do You Say to a Naked Lady?,1970,X,1970-02-18 00:00:00,85,"[Documentary, Comedy]",Allen Funt,[Allen Funt],"[Joie Addison, Laura Huston, Martin Meyers, Ka...",...,None,"[camera, naked, candid camera, elevator, react...","[Urs Furrer, Steve Karmen, Allen Funt, Tom Man...","[Allen Funt, Joie Addison, Laura Huston, Marti...",0,NaN,NaN,None,2,"[Joan Bell, Ernie Monah, Susanna Clemm, Donna ..."
3,tt0067741,Shaft,1971,R,1971-07-02 00:00:00,100,"[Action, Crime, Thriller]",Gordon Parks,"[Ernest Tidyman, John D.F. Black, Ernest Tidyman]","[Richard Roundtree, Moses Gunn, Charles Cioffi...",...,"[Metro Goldwyn Mayer, Shaft Productions Ltd.]","[new york city, black people, ghetto, daughter...","[Ernest Tidyman, Isaac Hayes, Gordon Parks, Jo...","[Richard Roundtree, Moses Gunn, Charles Cioffi...",4000000,NaN,NaN,"[[nm0005002, Isaac Hayes], [nm0005002, Isaac H...",1,"[Rex Robbins, George Strus, Joseph Leon, Marga..."
4,tt0068358,Charley-One-Eye,1973,R,1973-04-18 00:00:00+00:00,96,[Western],Don Chaffey,[Keith Leonard],"[Richard Roundtree, Roy Thinnes, Nigel Davenpo...",...,None,"[civil war, on the run, racism]","[James Swann, John Cameron, Don Chaffey, Tim H...","[Richard Roundtree, Roy Thinnes, Nigel Davenpo...",0,NaN,NaN,None,3,"[Rafael Albaicín, Luis Aller, Aldo Sambrell, J..."
